# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

In [207]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
mouse_study_combined = study_results.merge(mouse_metadata, how='left')

# Display the data table for preview
mouse_study_combined.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.0,0,Capomulin,Female,9,22
1,f932,0,45.0,0,Ketapril,Male,15,29
2,g107,0,45.0,0,Ketapril,Female,2,29
3,a457,0,45.0,0,Ketapril,Female,11,30
4,c819,0,45.0,0,Ketapril,Male,21,25


In [208]:
# Checking the number of mice.
number_of_mice = len(mouse_study_combined.groupby("Mouse ID")["Mouse ID"].unique())
number_of_mice

249

In [209]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
duplicate_mouse = mouse_study_combined[mouse_study_combined.duplicated()]
duplicate_mouse_id = duplicate_mouse["Mouse ID"].values
duplicate_mouse_id


array(['g989'], dtype=object)

In [210]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mouse_data = mouse_study_combined[mouse_study_combined.duplicated(subset=["Mouse ID", "Timepoint"], keep = False)]
duplicate_mouse_data

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
107,g989,0,45.000000,0,Propriva,Female,21,26
137,g989,0,45.000000,0,Propriva,Female,21,26
329,g989,5,48.786801,0,Propriva,Female,21,26
360,g989,5,47.570392,0,Propriva,Female,21,26
620,g989,10,51.745156,0,Propriva,Female,21,26
681,g989,10,49.880528,0,Propriva,Female,21,26
815,g989,15,51.325852,1,Propriva,Female,21,26
869,g989,15,53.442020,0,Propriva,Female,21,26
950,g989,20,55.326122,1,Propriva,Female,21,26
1111,g989,20,54.657650,1,Propriva,Female,21,26


In [211]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
#mouse_data_cleaned = mouse_study_combined.drop_duplicates(subset=["Mouse ID", "Timepoint"], keep="first")

mouse_study_combined.drop(mouse_study_combined.loc[mouse_study_combined['Mouse ID']==duplicate_mouse_id[0]].index, inplace=True)
mouse_study_combined

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
0,b128,0,45.000000,0,Capomulin,Female,9,22
1,f932,0,45.000000,0,Ketapril,Male,15,29
2,g107,0,45.000000,0,Ketapril,Female,2,29
3,a457,0,45.000000,0,Ketapril,Female,11,30
4,c819,0,45.000000,0,Ketapril,Male,21,25
...,...,...,...,...,...,...,...,...
1888,r944,45,41.581521,2,Capomulin,Male,12,25
1889,u364,45,31.023923,3,Capomulin,Male,18,17
1890,p438,45,61.433892,1,Ceftamin,Female,11,26
1891,x773,45,58.634971,4,Placebo,Female,21,30


In [214]:
# Checking the number of mice in the clean DataFrame.
len(mouse_study_combined.groupby("Mouse ID")["Mouse ID"].unique())

248

## Summary Statistics

In [232]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.

drug_mean = mouse_study_combined.groupby("Drug Regimen")["Tumor Volume (mm3)"].mean()
drug_variance = mouse_study_combined.groupby("Drug Regimen")["Tumor Volume (mm3)"].var()
drug_median = mouse_study_combined.groupby("Drug Regimen")["Tumor Volume (mm3)"].median()
drug_std = mouse_study_combined.groupby("Drug Regimen")["Tumor Volume (mm3)"].std()
drug_sem = mouse_study_combined.groupby("Drug Regimen")["Tumor Volume (mm3)"].sem()

stat_table = pd.DataFrame(
                        {"Mean Tumor Volume": drug_mean,
                        "Median Tumor Volume": drug_median,
                        "Tumor Volume Variance": drug_variance,
                        "Tumor Volume Std. Dev.": drug_std,
                        "Tumor Volume Std. Err.": drug_sem})
stat_table

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std. Dev.,Tumor Volume Std. Err.
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [236]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
mouse_study_combined.aggregate(func= 'mean')

TypeError: Could not convert ['b128f932g107a457c819h246p189n923q119f993z234b559y260x930o725z969v835r604n304l700x336l725m133v295a818y601t724k382w422c326c139v339a577y163k483k804o809z581a251i386c580q132u327v603f278g497d474o973c832o331m650v289m550h428r701v199x264f234c458q610j913a411a444d251j989y449k403c758x402r811a644i177g791a520u196m546w678n364s508e662z578r921a492w540v764z795e291e584e213j755s565a366p387b879i901k862g867s619w697j984c402h333k510p981t451a963m269z314o848v719q597c895a203f394c264n967f545k894k754g296d164w575x613q633b313f129j365p136c559a788s337q511m331o926d133n630a401w350c302a897j235q787a262r850i669n763s152c766e227c282v989w140v409l872o795y478q582s166x773w167t718o302i477c757t994p310a699k603x822l558l733f436l490b759l471y865y769l661j296u149u153w151h531a685o813m756n671s121v766w193e476w584b447k210a275x581n482t198l215s141o523i635w746r107s187g570x209x111z435n678g558u946o562v991c927x226p438b487o287q483t573f345i334l264j246r554s185b742x401l509s710r157u364j119v923w914i738r944y793g316g288l897w

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.


In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot


## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes

    
    # add subset 

    
    # Determine outliers using upper and lower bounds


In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
